<a href="https://colab.research.google.com/github/LuizGusQueiroz/Projetos_Estagio/blob/main/Folhas_de_pagamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

converte Folhas de pagamento recebidas em formato html para planilhas excell
registrando cada evento como uma linha

# Configuração do Ambiente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [348]:
import pandas as pd
import html
import os
from tqdm import tqdm

# Processamento Folha de Pagamento

## Arquivos html

In [ ]:
# Retorna ou salva em um arquivo .xlsx todos os registros de eventos
# da folha de pagamento recebida
def get_data_from_html(arq_html, nome_arq=''):
  start = 0
  end = bool()
  competencia    = str()
  empregador     = str()
  cod_empregado  = str()
  nome_empregado = str()
  cargo          = str()
  lotacao        = str()

  cols = ['Competência', 'Empregador', 'Código_Empregado', 'Nome_Empregado', 'Cargo', 'Lotação', 'Código_Evento', 'Desc_Evento', 'Provento', 'Desconto']
  data = pd.DataFrame(columns=cols)

  # Encontra a linha em que está a competência
  for row in arq_html:
    if 'de 2023' in row:
      competencia = get_competencia(row) # posso usar o get_content
      competencia = html.unescape(competencia)
      break

  # Encontra os dados de cada pessoa

  while True:

    # Encontra o empregador
    end = True
    for i, row in enumerate(arq_html[start:], start=start):
      if 'Empregador' in row:
        # Define onde irão começar os dados da pessoa
        end = False
        start = i
        # O nome do empregador está 2 linhas abaixo
        empregador = get_content(arq_html[i+2])
        break

    if end:
      break

    # Encontra o nome e código do empregado

    for i, row in enumerate(arq_html[start+1:], start=start+1):
      if 'Empregado' in row:
        # O código e nome do empregado está uma linha
        # abaixo da linha que tem 'Empregado'
        # os dados estão na forma 'codigo nome'
        dados_empregado = get_content(arq_html[i+1])
        # Separa os dados
        dados_split = dados_empregado.split()
        # O primeiro valor é o código
        cod_empregado = dados_split[0]
        # Os seguintes são o nome
        nome_empregado = ' '.join(dados_split[1:])
        break

    # Encontra o cargo

    for i, row in enumerate(arq_html[start+1:], start=start+1):
      if 'Cargo' in row:
        # O cargo está duas linhas abaixo da
        # linha que tem 'Cargo'
        cargo = get_content(arq_html[i+2])
        # Ajusta à formatação html
        cargo = html.unescape(cargo)
        break

    # Encontra a lotação

    for i, row in enumerate(arq_html[start+1:], start=start+1):
      if 'Lota&ccedil;&atilde;o' in row:
        lotacao = get_content(arq_html[i+2])
        # Ajusta à formatação html
        lotacao = html.unescape(lotacao)
        break

    # Guarda os eventos
    iterador = 0
    while True:
      try:
        eventos = get_eventos(arq_html, start, iterador)
        break
      except:
        iterador +=1
        if iterador == 10:
          break

    # Salva os registros por evento

    for i in range(eventos.shape[0]):
      registro = [competencia, empregador, cod_empregado, nome_empregado, cargo, lotacao]
      if eventos['Código'].iloc[i]:
        registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Provento'].iloc[i], eventos['Desconto'].iloc[i]])
        data.loc[len(data)] = registro

    start = start + 10

  data['Desc_Evento'] = data['Desc_Evento'].apply(html.unescape)

  competencia = competencia.replace(' de ', '')
  nome_arq = f'{nome_arq}_{competencia}.xlsx'
  data.to_excel(nome_arq, index=False)

  return data

In [ ]:
def get_competencia(row):
  start = int()
  end = int()
  index = row.find('2023')

  for i in range(index, 0, -1):
    if row[i] == '>':
      start = i+1
      break

  for i in range(index, len(row)):
    if row[i] == '<':
      end = i
      break

  return row[start:end]

In [ ]:
def get_content(row):
  start = int()
  end = int()
  index = row.find('class')

  start = row.find('>', index) + 1
  end = row.find('<', index)

  return row[start:end]

In [ ]:
def get_eventos(html, start, iterador):
  # As linhas que antecedem o começo e fim dos eventos seguem os modelos
  # Neste modelo, a classe varia entre f0 até f4, então um laço testa para estes valores
  modelo_start = f'<font class="f{iterador}"><b>Cod.</b></font></div>'
  modelo_end = f'<font class="f{iterador}">Total de Proventos</font></div>'


  start_event = int()
  end_event = int()
  # Encontra o index do início dos eventos
  for i, row in enumerate(html[start:], start=start):
    if modelo_start in row:
      start_event = i
      break
  # Encontra o index do fim dos eventos
  for i, row in enumerate(html[start:], start):
    if modelo_end in row:
      end_event = i
      break

  dados = list()
  value = list()
  contador = 0

  for row in html[start_event+1:end_event]:
    if 'class' in row:
      if contador < 5:
        value.append(get_content(row))
        contador += 1
      else:
        contador = 1
        dados.append(value)
        value = list()
        value.append(get_content(row))
  dados.append(value)

  df = pd.DataFrame(dados)
  df = df[list(df.columns[::-1])]


  df.columns = ['Código', 'Descrição', 'Referência', 'Provento', 'Desconto']

  return df


## Arquivos xlsx

In [ ]:
def get_data_from_xlsx(path, competencia, empregador, start, end, indice):
  data = pd.DataFrame(columns=['Competência',	'Empregador',	'Código_Empregado',	'Nome_Empregado',	'Cargo',	'Lotação',	'Código_Evento',	'Desc_Evento',	'Provento',	'Desconto'])

  columns = ['Numero', 'Nome', 'Jorn.', 'Cargo', 'Admissão', 'SF', 'IR']
  nrows = end - start + 2
  arq = pd.read_excel(path, skiprows=start, nrows=nrows).fillna('')
  arq.columns = columns

  nome = str()
  codigo = str()
  cargo = str()
  index = 1
  registro = list()

  lotacao = arq['Numero'].iloc[0]
  # Percorre todos os usuários, o fim dos dados de cada usuários
  # é seguido do código do próximo
  while arq['Numero'].iloc[index].isnumeric():
    eventos = pd.DataFrame(columns=['Código', 'Descrição', 'Provento', 'Desconto'])

    codigo = arq['Numero'].iloc[index]
    nome = arq['Nome'].iloc[index]
    cargo = arq['Cargo'].iloc[index]
    # Encontra o começo dos eventos
    for i in range(index, nrows):
      if arq['Numero'].iloc[i] == 'Eventos':
        index = i+1
        break

    # Este while coleta todos os eventos
    # Os eventos são encerrados pela palavra 'Totais'
    while arq['Numero'].iloc[index] != 'Totais':
      registro = list()
      # Codigo evento
      registro.append(arq['Numero'].iloc[index])
      # Descrição
      registro.append(arq['Nome'].iloc[index])
      # Provento
      registro.append(arq['Admissão'].iloc[index])
      # Desconto
      registro.append(arq['SF'].iloc[index])

      eventos.loc[len(eventos)] = registro
      index += 1
    index += 1

    #Para cada linha de eventos, cria um registro completo
    for i in range(eventos.shape[0]):
      registro = list()
      registro.append(competencia)
      registro.append(empregador)
      registro.append(codigo)
      registro.append(nome)
      registro.append(cargo)
      registro.append(lotacao)
      registro.append(eventos['Código'].iloc[i])
      registro.append(eventos['Descrição'].iloc[i])
      registro.append(eventos['Provento'].iloc[i])
      registro.append(eventos['Desconto'].iloc[i])

      data.loc[len(data)] = registro
  nome_arq = f'folha_parte_{indice:04}.xlsx'
  data.to_excel(nome_arq, index=False)

  #return data

In [ ]:
def fix_cod(cod):
  new_cod = f'{cod:04}'
  return new_cod

# Leitura dos arquivos

## Arquivos html

In [ ]:
nome_a_salvar = 'GA_'
nome_arquivo = '<MES>.html'
primeiro_mes = 10
ultimo_mes = 11


ultimo_mes += 1
s = primeiro_mes
t = ultimo_mes

In [ ]:
for i in tqdm(range(s, t)):
  mes = f'{i:02}'
  path = f'/content/drive/My Drive/GA/{nome_arquivo}'.replace('<MES>', mes)
  with open(path, 'r') as file:
      html = file.read()
  html = html.split('\n')

  get_data_from_html(html, nome_a_salvar)

100%|██████████| 2/2 [00:05<00:00,  2.86s/it]


### Complementar

In [ ]:
# Caso haja complementar
mes = '06'
path = f'/content/drive/My Drive/HISEG/TCM/062023 COMPLEMENTAR.html'
with open(path, 'r') as file:
    html = file.read()
html = html.split('\n')

get_data(html, 'TCM_COMPLEMENTAR_')

## Arquivos xlsx

In [345]:
empregador = '8498 FIBRA TEC'
competencia = 'Novembro de 2023'
mes = '11'

path =f'/content/drive/My Drive/FIBRA/{empregador}/{mes}2023.xls'
data = pd.read_excel(path).fillna('')

# Define os intervalos de cada Lotação
indices = [1]
for i in range(data.shape[0]):
  if data['Numero'].iloc[i] == 'Totais':
    if not data['Numero'].iloc[i+1].isnumeric():
      if 'Página de 1 de 1' not in data['Numero'].iloc[i+1]:
        indices.append(i+1)
indices.append(data.shape[0])
empregador = data['Numero'].iloc[0]

for i in tqdm(range(len(indices)-1)):
  get_data_from_xlsx(path, competencia, empregador, indices[i], indices[i+1], i)

100%|██████████| 51/51 [00:09<00:00,  5.62it/s]


In [ ]:
files = os.listdir()
files.sort()
df = pd.DataFrame()
for file in tqdm(files):
  if 'folha_parte_' in file:
    df_temp = pd.read_excel(file)
    df = pd.concat([df, df_temp], axis=0)

df['Código_Evento'] = df['Código_Evento'].apply(fix_cod)
nome_arq = f'Folha_FIBRA{empregador.split()[0]}_{competencia.split()[0]}.xlsx'
df.to_excel(nome_arq, index=False)

# Junta Planilhas

Concatena planilhas agrupadas por mês
e no final cria planilhas com os valores únicos das colunas
['Empregador', 'Código_Empregado', 'Nome_Empregado', 'Cargo', 'Lotação']
agrupados por mês também

In [ ]:
#diretorio = '/content/drive/My Drive/FolhasPagamento'
#files = os.listdir(diretorio)

nome_empresa = 'GA'
files = os.listdir()

ano = '2023'
meses = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro']
meses = ['Outubro', 'Novembro']
meses = [f'{mes}{ano}' for mes in meses]

colunas = ['Empregador', 'Código_Empregado', 'Nome_Empregado', 'Cargo', 'Lotação']

for mes in tqdm(meses):
  arquivos = list()
  df_mensal = pd.DataFrame()

  for file in files:
    if mes in file:
      arquivos.append(file)
      #arquivos.append(f'{diretorio}/{file}')

  for arq in arquivos:
    df = pd.read_excel(arq)
    df_mensal = pd.concat([df_mensal, df], axis=0)

  # Coleta os registros únicos das colunas desejadas
  for coluna in colunas:

    unique_values = df_mensal[coluna].unique()
    unique_values = pd.DataFrame(unique_values)
    unique_values.to_excel(f'{coluna}_unicos_{mes[:3]}.xlsx', index=False)

  df_mensal.to_excel(f'{nome_empresa}_{mes}.xlsx', index=False)


100%|██████████| 2/2 [00:00<00:00,  2.22it/s]


In [ ]:
# Junta de grupos diferentes

diretorio = '/content/drive/My Drive/Registros Acumulados Totais/Janeiro/'
files = os.listdir(diretorio)

df = pd.DataFrame()

for file in tqdm(files):
  df_temp = pd.read_excel(diretorio+file)
  df = pd.concat([df, df_temp], axis=0)

df.to_excel('Registros_acumulados_Janeiro.xlsx', index=False)

# Limpa arquivos desnecessários

In [365]:
def clean():
  for file in os.listdir():
    if '.xlsx' in file or '.html' in file:
      os.remove(file)